# 01 — Exploration

## Goals
- Run the EDA pipeline to profile the session and customer data.
- Produce a reusable `artifacts/eda` folder containing the report, cleaned data, and metadata.

## Inputs
- `config/eda.yaml` (cohort definition, extraction filters, cleaning rules, report settings).

## Outputs
- `artifacts/eda/<timestamp>/` with EDA artifacts (report, metadata, parquet files).
- `artifacts/eda/latest/` symlink or copy pointing to the most recent run.

> The base output directory is deterministic (`artifacts/eda`) so reruns are consistent for CI and reviewers.


In [ ]:
from pathlib import Path

from traveltide.eda import run_eda

base_outdir = Path("artifacts/eda")
base_outdir.mkdir(parents=True, exist_ok=True)

run_dir = run_eda(config_path="config/eda.yaml", outdir=str(base_outdir))
run_dir
